In [11]:
## Installing the polars gpu engine

!pip install -U polars[gpu] --extra-index-url=https://pypi.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Using cached polars-1.27.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
INFO: pip is looking at multiple versions of cudf-polars-cu12 to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 52.4 MB/s eta 0:00:00
  Using cached polars-1.27.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
INFO: pip is still looking at multiple versions of cudf-polars-cu12 to determine which version is compatible with other requirements. This could take a while.
  Using ca

In [12]:
!pip install hvplot jupyter_bokeh holoviews==1.19 pynvml ## visualization capabilities of polar

In [13]:
import pynvml
pynvml.nvmlInit()
pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0))
mem = pynvml.nvmlDeviceGetMemoryInfo(pynvml.nvmlDeviceGetHandleByIndex(0))
mem = mem.total/1e9
if mem < 24:
    !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet -O transactions.parquet
else:
    !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions.parquet -O transactions.parquet

!wget https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv

--2025-04-13 17:07:23--  https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.207, 172.253.117.207, 142.250.99.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857658422 (818M) [application/octet-stream]
Saving to: ‘transactions.parquet’

transactions.parque 100%[===================>] 817.93M   135MB/s    in 5.9s    

2025-04-13 17:07:29 (138 MB/s) - ‘transactions.parquet’ saved [857658422/857658422]

--2025-04-13 17:07:29--  https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.207, 172.253.117.207, 142.250.99.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131421 (128K) [t

In [14]:
import polars as pl
import hvplot.polars # Using the hvplot as the plotting backend
from polars.testing import assert_frame_equal

In [15]:
pl.__version__ # Make sure the version is >= `1.6.0`; If not restart the session.

'1.25.2'

In [16]:
transactions = pl.scan_parquet("transactions.parquet")

In [17]:
transactions.collect_schema()

Schema([('CUST_ID', String),
        ('START_DATE', Date),
        ('END_DATE', Date),
        ('TRANS_ID', String),
        ('DATE', Date),
        ('YEAR', Int64),
        ('MONTH', Int64),
        ('DAY', Int64),
        ('EXP_TYPE', String),
        ('AMOUNT', Float64)])

In [18]:
transactions.head(5).collect()

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
str,date,date,str,date,i64,i64,i64,str,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06


In [19]:
%%time
transactions.select(pl.col("AMOUNT").sum()).collect()

CPU times: user 1.37 s, sys: 429 ms, total: 1.8 s
Wall time: 1.03 s


AMOUNT
f64
3.6183e9


In [21]:
%%time
transactions.select(pl.col("AMOUNT").sum()).collect(engine="gpu")

CPU times: user 164 ms, sys: 44.1 ms, total: 208 ms
Wall time: 173 ms


AMOUNT
f64
3.6183e9


In [22]:
gpu_engine = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=True, # Fail loudly if we can't run on the GPU.
)

In [23]:
%%time
transactions.select(pl.col("AMOUNT").sum()).collect(engine=gpu_engine)

CPU times: user 159 ms, sys: 51.3 ms, total: 210 ms
Wall time: 177 ms


AMOUNT
f64
3.6183e9


In [24]:
%%time

res_cpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect()
)
res_cpu

CPU times: user 4.39 s, sys: 3.25 s, total: 7.63 s
Wall time: 5.11 s


CUST_ID,AMOUNT
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In [25]:
%%time

res_cpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect(engine="gpu")
)
res_cpu

CPU times: user 272 ms, sys: 94.7 ms, total: 367 ms
Wall time: 570 ms


CUST_ID,AMOUNT
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In [26]:
%%time

res_cpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect(engine=gpu_engine)
)
res_cpu

CPU times: user 287 ms, sys: 61.4 ms, total: 348 ms
Wall time: 324 ms


CUST_ID,AMOUNT
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In [27]:
query = """
SELECT CUST_ID, SUM(AMOUNT) as sum_amt
FROM transactions
GROUP BY CUST_ID
ORDER BY sum_amt desc
LIMIT 5
"""

%time pl.sql(query).collect()
%time pl.sql(query).collect(engine=gpu_engine)

CPU times: user 4.57 s, sys: 2.59 s, total: 7.16 s
Wall time: 4.63 s
CPU times: user 265 ms, sys: 82.7 ms, total: 348 ms
Wall time: 299 ms


CUST_ID,sum_amt
str,f64
"""CA9UYOQ5DA""",2.0290e6
"""CJUK2MTM5Q""",1.8115e6
"""CYXX1NBIKL""",1.8082e6
"""C6ILEYAYQ9""",1.7961e6
"""CCNBC305GI""",1.7274e6


In [28]:
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect()
)

CPU times: user 4.57 s, sys: 2.57 s, total: 7.14 s
Wall time: 5.43 s


CUST_ID,max_amount
str,f64
"""CIP0I11MG2""",6201.45
"""C4O38N5TQS""",6077.49
"""CL2M3N3K90""",6041.59
"""CC472PU9O8""",5929.14
"""CGR8UI27OK""",5903.61


In [29]:
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect(engine=gpu_engine)
)

CPU times: user 249 ms, sys: 57.5 ms, total: 307 ms
Wall time: 416 ms


CUST_ID,max_amount
str,f64
"""CIP0I11MG2""",6201.45
"""C4O38N5TQS""",6077.49
"""CL2M3N3K90""",6041.59
"""CC472PU9O8""",5929.14
"""CGR8UI27OK""",5903.61


In [30]:

names = ['Location', 'Rainfall (inches)', 'Date', 'YEAR', 'MONTH', 'DAY']

weather = pl.scan_csv("rainfall_data_2010_2020.csv", new_columns=names)
weather.head().collect()

Location,Rainfall (inches),Date,YEAR,MONTH,DAY
str,f64,i64,i64,i64,i64
"""Tatooine""",0.33,20100101,2010,1,1
"""Tatooine""",0.0,20100102,2010,1,2
"""Tatooine""",0.28,20100103,2010,1,3
"""Tatooine""",0.26,20100104,2010,1,4
"""Tatooine""",0.39,20100105,2010,1,5


In [31]:
weather_cleaned = (
    weather
    .with_columns(pl.col("Date").cast(pl.Utf8).str.strptime(pl.Date(), "%Y%m%d"))
    .collect()
)

In [32]:
%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect()
)

CPU times: user 11.8 s, sys: 4.23 s, total: 16.1 s
Wall time: 10.1 s


EXP_TYPE,DATE,Rainfall (inches)
str,date,f64
"""Bills and Utilities""",2010-01-01,0.33
"""Clothing""",2010-01-01,0.33
"""Education""",2010-01-01,0.33
"""Entertainment""",2010-01-01,0.33
"""Fines""",2010-01-01,0.33


In [33]:
%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect(engine=gpu_engine)
)

CPU times: user 462 ms, sys: 96.9 ms, total: 559 ms
Wall time: 717 ms


EXP_TYPE,DATE,Rainfall (inches)
str,date,f64
"""Bills and Utilities""",2010-01-01,0.33
"""Clothing""",2010-01-01,0.33
"""Education""",2010-01-01,0.33
"""Entertainment""",2010-01-01,0.33
"""Fines""",2010-01-01,0.33


In [34]:
res = (
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.sum("Rainfall (inches)"), pl.sum("AMOUNT"))
    .sort(["YEAR", "MONTH"])
    .collect(engine=gpu_engine)
)

res

EXP_TYPE,YEAR,MONTH,Rainfall (inches),AMOUNT
str,i64,i64,f64,f64
"""Health""",2010,1,51.45,12852.68
"""Groceries""",2010,1,294.3,26341.02
"""Bills and Utilities""",2010,1,59.41,26307.97
"""Fines""",2010,1,0.91,315.2
"""Education""",2010,1,25.67,20602.49
…,…,…,…,…
"""Fines""",2020,12,3.43,13707.66
"""Clothing""",2020,12,413.24,5.5658e6
"""Motor/Travel""",2020,12,1434.55,1.5570e7


In [35]:
engine_with_fallback = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=False, # Fallback to CPU if we can't run on the GPU (this is the default)
)

In [36]:
%%time

result = transactions.with_columns(
    pl.col('AMOUNT').rolling_mean(
        window_size=7
    )
     .alias('rolling_avg')
).collect(engine=engine_with_fallback)

result.head()

CPU times: user 6.29 s, sys: 3.37 s, total: 9.66 s
Wall time: 6.89 s


CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT,rolling_avg
str,date,date,str,date,i64,i64,i64,str,f64,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27,null
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85,null
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8,null
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72,null
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06,null


In [37]:
(
    res
    .with_columns(
        pl.date(pl.col("YEAR"), pl.col("MONTH"), 1).alias("date-month"),
        pl.col("Rainfall (inches)")*100,
    )
    .hvplot.line(
        x="date-month", y=["AMOUNT", "Rainfall (inches)"],
        by=['EXP_TYPE'],
        rot=45,
    )
)

:NdOverlay   [Variable]
   :Curve   [date-month]   (value)